In [ ]:
# Instalacja bibliotek
!pip install pandas matplotlib textblob transformers torch

# Importowanie bibliotek
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
from transformers import pipeline
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter
nltk.download('stopwords')

# Przykładowe dane
data = {
    'text': [
        "I love this new phone, it's amazing!",
        "Terrible service, I'm so disappointed.",
        "The movie was okay, nothing special.",
        "Fantastic weather today, feeling great!",
        "This product is a total waste of money.",
        "Pretty good experience overall."
    ]
}
df = pd.DataFrame(data)
print("Przykładowe dane:")
print(df)

# Preprocessing
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^A-Za-z\s]', '', text)
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

df['cleaned_text'] = df['text'].apply(clean_text)
print("\nDane po preprocessingu:")
print(df)

# Analiza sentymentu
def get_textblob_sentiment(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    if polarity > 0:
        return 'Positive'
    elif polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

df['textblob_sentiment'] = df['cleaned_text'].apply(get_textblob_sentiment)
sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')
df['hf_sentiment'] = df['text'].apply(lambda x: sentiment_pipeline(x)[0]['label'].capitalize())
print("\nDane z analizą sentymentu:")
print(df)

# Wizualizacja
plt.figure(figsize=(8, 5))
df['textblob_sentiment'].value_counts().plot(kind='bar', color=['green', 'red', 'blue'])
plt.title('Rozkład sentymentu (TextBlob)')
plt.xlabel('Sentyment')
plt.ylabel('Liczba postów')
plt.show()

plt.figure(figsize=(8, 5))
df['hf_sentiment'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.title('Rozkład sentymentu (Hugging Face)')
plt.xlabel('Sentyment')
plt.ylabel('Liczba postów')
plt.show()

# Analiza słów kluczowych
all_text = ' '.join(df['cleaned_text'])
words = all_text.split()
word_freq = Counter(words).most_common(10)
print("\nNajczęściej występujące słowa:")
for word, freq in word_freq:
    print(f"{word}: {freq}")

words_df = pd.DataFrame(word_freq, columns=['word', 'frequency'])
plt.figure(figsize=(10, 6))
words_df.plot(kind='bar', x='word', y='frequency', legend=False, color='skyblue')
plt.title('Najczęściej występujące słowa')
plt.xlabel('Słowo')
plt.ylabel('Częstotliwość')
plt.xticks(rotation=45)
plt.show()

# Interfejs tekstowy
user_text = input("\nWpisz tekst do analizy sentymentu: ")
cleaned_user_text = clean_text(user_text)
textblob_result = get_textblob_sentiment(cleaned_user_text)
hf_result = sentiment_pipeline(user_text)[0]['label'].capitalize()
print(f"TextBlob: {textblob_result}")
print(f"Hugging Face: {hf_result}")